<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Required-libraries" data-toc-modified-id="Required-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Required libraries</a></span></li><li><span><a href="#Print-Date" data-toc-modified-id="Print-Date-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Print Date</a></span></li><li><span><a href="#CSV-files-and-DataFrames" data-toc-modified-id="CSV-files-and-DataFrames-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>CSV files and DataFrames</a></span><ul class="toc-item"><li><span><a href="#Declare-known-data-files" data-toc-modified-id="Declare-known-data-files-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Declare known data files</a></span></li><li><span><a href="#Function:-Fetch-DataFrame-from-CSV-file¶" data-toc-modified-id="Function:-Fetch-DataFrame-from-CSV-file¶-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Function: Fetch DataFrame from CSV file¶</a></span></li></ul></li><li><span><a href="#DataFrame-Transformations" data-toc-modified-id="DataFrame-Transformations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>DataFrame Transformations</a></span><ul class="toc-item"><li><span><a href="#Shift-Date-by-X-days-in-DataFrame" data-toc-modified-id="Shift-Date-by-X-days-in-DataFrame-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Shift Date by X days in DataFrame</a></span></li></ul></li><li><span><a href="#Rata-Die" data-toc-modified-id="Rata-Die-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Rata Die</a></span><ul class="toc-item"><li><span><a href="#First-Day" data-toc-modified-id="First-Day-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>First Day</a></span></li><li><span><a href="#Function:-Days-Since-Day-One" data-toc-modified-id="Function:-Days-Since-Day-One-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Function: Days Since Day One</a></span></li><li><span><a href="#Function:-Update-Rada-Die-in-DataFrame" data-toc-modified-id="Function:-Update-Rada-Die-in-DataFrame-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Function: Update Rada Die in DataFrame</a></span></li><li><span><a href="#Insert-Rata-Die-into-DataFrame" data-toc-modified-id="Insert-Rata-Die-into-DataFrame-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Insert Rata Die into DataFrame</a></span></li></ul></li></ul></div>

# Required libraries

In [1]:
# import Pkg; Pkg.add("CSV")
using CSV

# import Pkg; Pkg.add("Dates")
using Dates

# import Pkg; Pkg.add("DataFrames")
using DataFrames

# Print Date

In [16]:
# https://docs.julialang.org/en/v1/stdlib/Dates/#Dates.now-Tuple{}
# https://github.com/JuliaLang/julia/issues/37480

function updated() 
    println( "Last update: ", Dates.format(Dates.now(), "u. d, yyyy HH:MM"))
end

updated (generic function with 1 method)

# CSV files and DataFrames

## Declare known data files

In [3]:
file_ISM_Mfc_Emp = "data/united-states.ism-manufacturing-employment.csv"
file_Markkit_Mfc_PMI = "data/united-states.markit-manufacturing-pmi.csv"
file_initial_jobless_claims = "data/united-states.initial-jobless-claims.csv"

"data/united-states.markit-manufacturing-pmi.csv"

## Function: Fetch DataFrame from CSV file¶

In the future, I expect to have large data sets that are too big for the GitHub, so I am planning for the separate downlod location i.e. Google Drive, etc.

In [27]:
function fetch_data(file_path, date_format="yyyy-mm-dd")
    isfile(file_path) || # does file exist locally?
        download( string("https://raw.githubusercontent.com/UkiDLucas/MarketIndicators.jl/master/", 
            file_path), # URL
        file_path) # save it as name
    
    return CSV.read(        # returns DataFrame
        file_path, 
        dateformat="$date_format"
        )
end

println("function: df = fetch_data(file_path, date_format=\"yyyy-mm-dd\" )")

function: df = fetch_data(file_path, date_format="yyyy-mm-dd" )


In [6]:
# TEST
#df = fetch_data(file_ISM_Mfc_Emp, "yyyy.mm.dd")

# DataFrame Transformations

## Shift Date by X days in DataFrame

In [28]:
function shift_dates_by_days!(df::DataFrame, shift_days::Int64=-1, date_column::Int64=1)
    rows = size(df)[1]
    
    for row in 1:rows
        original = Date(df[row, date_column]) 
        shifted = original + Dates.Day(shift_days)
        # println("original ", original, " ", shift_days, " shift ", shifted)
        df[row, date_column] = shifted
    end
    return df
end

# df = shift_dates_by_days!(df3, -90)
println("function: df = shift_dates_by_days!(df, shift_days=-1, date_column=1)")

function: df = shift_dates_by_days!(df, shift_days=-1, date_column=1)


# Rata Die

## First Day

In [8]:
first_date = Date("Jan. 1, 1900", "u. dd, yyyy")

1900-01-01

In [9]:
day_one = Dates.datetime2rata(first_date) # typeof(rate_die) = Int64

693596

## Function: Days Since Day One

In [29]:
function days_since_day_one(df, row, date_column) 
    date = df[row, date_column] # type Date
    rata_die = Dates.datetime2rata(date) 
    rata_die = rata_die - day_one
    return rata_die
end

println("function: rata_die = days_since_day_one(df, row, date_column) ")

function:  rata_die = days_since_day_one(df, row, date_column) 


## Function: Update Rada Die in DataFrame

In [30]:
function update_rata_die!(df, days_column=1, date_column=2)
    rows = size(df)[1] # first part of the returned tupple

    for row = 1:rows
        days = days_since_day_one(df, row, date_column)
        # println(days)
        df[row, days_column] = days
    end
    return df
end

println("function: df = update_rata_die!(df, days_column=1, date_column=2)")

function: df = update_rata_die!(df, days_column=1, date_column=2)


## Insert Rata Die into DataFrame

In [32]:
function insert_rata_die_column!(df)
    rows = size(df)[1] # first number of the tuple
    # https://stackoverflow.com/a/63731422/6312771
    insertcols!(df, 1,  :Day => 1:rows, makeunique=true) # insert as column 1, populate with 1,2,3,..
    return df
end
println("function: df = insert_rata_die_column!(df)")

function: df = insert_rata_die_column!(df)
